## Webscrape for Data Related Jobs in Canada
----
4 queries were made into ca.Indeed.com to obtain job postings for (1) Chemical Engineer, (2) Process Engineer, (3) Process Design Engineer, (4) Clean Energy Engineer, (5) Renewable Energy Engineer, (6) Data Analyst Engineer, (7) Mettallurgical Engineer, (8) Metallurgy engineer. Data will be limited to the last 14 days. Due to large quantities of job postings, a limit of 70 pages were extracted (equaling around 1000 job posts) for each role. 

Job title index was also assigned to each job posting during the web scrape. For example, while scraping for Data Analyst roles, an index number of 1 was assigned to each posting. This will help for analysis.

In [ ]:
# ! pip install fake_useragent

In [24]:
# Dependencies
from bs4 import BeautifulSoup as Soup
import requests
import pymongo
import urllib, requests, re, pandas as pd
from pprint import pprint
import random
from fake_useragent import UserAgent
import time
from selenium import webdriver

In [25]:
# From here we generate a random user agent
ua = UserAgent()

In [26]:
# Get a list of latest proxies from a free website
res = requests.get('https://www.sslproxies.org/', headers={'User-Agent':'Mozilla/5.0'})
soup = Soup(res.text,"lxml")

proxy_list = []

for items in soup.select("#proxylisttable tbody tr"):
    proxy_ip = ':'.join([item.text for item in items.select("td")[:2]])
    proxy_list.append(proxy_ip)

print(proxy_list)

['118.173.232.61:47115', '36.72.104.210:8080', '87.140.8.148:8080', '41.190.95.20:56167', '165.22.81.30:46215', '60.246.7.4:8080', '103.241.227.107:6666', '195.169.35.228:8080', '51.81.82.175:80', '162.127.140.112:8080', '14.251.159.15:4145', '139.255.11.147:8080', '37.15.208.106:53281', '191.97.38.193:23500', '172.104.4.99:3128', '208.80.28.208:8080', '18.135.32.174:80', '195.25.19.4:8080', '131.108.118.27:8080', '87.183.133.143:8080', '103.152.5.70:8080', '181.129.183.19:53281', '52.175.120.142:8080', '195.209.51.61:11978', '187.45.123.137:36559', '35.182.149.39:3129', '118.27.28.45:3128', '118.172.181.147:34388', '79.99.18.86:3128', '106.104.148.2:80', '104.238.81.186:56221', '101.108.119.192:8080', '110.74.203.249:8080', '177.22.18.236:3128', '177.4.173.226:80', '180.243.129.184:8080', '139.59.122.99:8118', '27.145.54.247:80', '134.236.150.126:8080', '81.12.28.99:8080', '171.97.34.170:8080', '188.186.180.135:8080', '18.228.17.224:19940', '187.86.158.117:3128', '45.174.92.2:999', '1

In [27]:
# list job titles to be searched
titles = ["Process+EIT",
          "Process+Engineer-in-training",         
          "Junior+Process+Engineer",
          "Junior+Chemical+Engineer",
          "Junior+Project+Engineer",
          "Junior+R&D+Engineer",
          "Junior+Field+Production+Engineer",
          "Junior+Reliability+Engineer",
          "Junior+QA+Engineer"
          "Junior+Water+Engineer",
          "Junior+Data+Engineer",
          "Junior+Sustainable+Energy+Engineer",
          "Junior+Renewable+Energy+Engineer",         
          "Junior+Mettallurgical+Engineer"
          ]

# Create empty lists to collect information later
job_title_list = []
job_title_index = []
company_list = []
job_id_list = []
location_list = []
links_list = []

In [28]:
# Create random parameters
user_agent = ua.random
header = {"user-agent": str(user_agent)}

# Create random proxy list
proxy = random.choice(proxy_list)
proxy_protocol = {
    "http"  : proxy,
    "https" : proxy
}

In [29]:
# country_code = "ca"
# country = "canada"
# days = "14"
# title = "Process Engineer"
# page = 0



# url = f'https://{country_code}.indeed.com/jobs?q={title}&l={country}&sort=date&fromage={days}&start={page}'

# # Retrieve page with the requests module
# response = requests.get(
#                     url,
#                     #proxies=proxy_protocol,
#                     headers=header
#                 )

# # Create BeautifulSoup object; parse with 'html.parser'
# soup = Soup(response.text, 'lxml')
           
# # Retrieve the parent divs for all articles
# results = soup.find_all('div', class_='result')

# for result in results:
            
#     try:
#         job_link = result.find('a', class_='turnstileLink').get('href')
#         click_link = f"https://{country_code}.indeed.com{job_link}"

#     except:
#         pass
    
# print(click_link)

In [30]:
# Country code 
country_code = "ca"
country = "canada"
days = "14"

# Start the main web scraping
for i, title in enumerate(titles):

    print("---------------")
    print("Starting job search for: ", title)
    # Reset page to 0
    page = 0
    counter = True

    while counter == True:
        
        # search query for Data Analyst roles
        url = f'https://{country_code}.indeed.com/jobs?q={title}&l={country}&sort=date&fromage={days}&start={page}'
        print("Current page: ", url)

        # Random time gap
        time_gap = random.randrange(3, 7, 1)
        time.sleep(time_gap)
        
        # Retrieve page with the requests module
        response = requests.get(
                            url,
                            #proxies=proxy_protocol,
                            headers=header
                        )
        
        # Create BeautifulSoup object; parse with 'html.parser'
        soup = Soup(response.text, 'lxml')
           
        # Retrieve the parent divs for all articles
        results = soup.find_all('div', class_='result')


        # For page one, calculate the page number by deviding job counts by 15 (each Indeed page has 15 postings)
        if page == 0:
            
            try:
            
                job_count = soup.find('div', id='searchCountPages').text.strip()
                job_count = job_count.replace(",", "")
                job_count = int(job_count.split(" ")[3])
                page_count = round(job_count / 15, 0)
                page_range = int(page_count)

                if page_count == 0:
                    page_count = 1
                    page_range = int(page_count)
                    
                    
                print("Page number: ", int(page_count))

            except:
                counter = False
                print("There is 0 result for this job title")
                print("---------------")

        # Stop going to the next page when the last page was reached, 10 is because page goes as 10,20,30,...
        elif page == page_range*10:
            counter = False


        # loop over results to get article data
        for result in results:
            
            try:
                # scrape the article header 
                job_title = result.find('a', class_='jobtitle').text.strip()
                job_link = result.find('a', class_='jobtitle').get('href')

                # give the current title an index for differentiation purpose later in analysys step
                job_index = i + 1

                # scrape information
                company = result.find('span', class_='company').text.strip()
                job_id = result.get('id')
                location = result.find(class_='location').text 
                job_link = result.find('a', class_='turnstileLink').get('href')
                click_link = f"https://{country_code}.indeed.com{job_link}"
                
                # append to the lists
                job_title_list.append(job_title)
                job_title_index.append(job_index)
                company_list.append(company)
                location_list.append(location)
                job_id_list.append(job_id)
                links_list.append(click_link)
                
            except:
                pass

        # Update page parameter by adding 10
        page += 10

        # Every 10 pages, get random UA
        if page % 100 == 0:

            user_agent = ua.random
            header = {"user-agent": str(user_agent)}
            print(f"---------------\n\
                A new user-agent was created:\n\
                {user_agent}\n----------------")

    


print("===================\nScraping completed")

---------------
Starting job search for:  Process+EIT
Current page:  https://ca.indeed.com/jobs?q=Process+EIT&l=canada&sort=date&fromage=14&start=0
Page number:  2
Current page:  https://ca.indeed.com/jobs?q=Process+EIT&l=canada&sort=date&fromage=14&start=10
Current page:  https://ca.indeed.com/jobs?q=Process+EIT&l=canada&sort=date&fromage=14&start=20
---------------
Starting job search for:  Process+Engineer-in-training
Current page:  https://ca.indeed.com/jobs?q=Process+Engineer-in-training&l=canada&sort=date&fromage=14&start=0
Page number:  1
Current page:  https://ca.indeed.com/jobs?q=Process+Engineer-in-training&l=canada&sort=date&fromage=14&start=10
---------------
Starting job search for:  Junior+Process+Engineer
Current page:  https://ca.indeed.com/jobs?q=Junior+Process+Engineer&l=canada&sort=date&fromage=14&start=0
Page number:  1
Current page:  https://ca.indeed.com/jobs?q=Junior+Process+Engineer&l=canada&sort=date&fromage=14&start=10
---------------
Starting job search for: 

In [31]:
df = pd.DataFrame({
                "Job Title Index" : job_title_index,    
                "Job ID" : job_id_list,
                "Job Title" : job_title_list, 
                "Company Name" : company_list, 
                "Company Location" : location_list,
                "Link": links_list
                    
})


df.to_csv('CA_chemengjobs.csv')